# Skyrmion in Pd/Fe/Ir(111)

In this tutorial we will reproduce Malottki et. al (Enhanced skyrmion stability due to exchange frustration
JO  - Scientific Reports 2017, https://rdcu.be/d9rHc)

In this article Pd/Fe/Ir(111) layers are studied and Heisenberg interaction parameters are obtained from DFT for hcp and fcc stacking, respectively. The interaction parameters (*Table 1* in Malottki et. al) are:

### Interaction parameters

**Tab. 1:** Interaction parameters as per Malottki et. al

|         | J1    | J2    | J3    | J4   | J5   | J6   | J7   | J8    | J9    | D1   | K    |
|---------|-------|-------|-------|------|------|------|------|-------|-------|------|------|
| **hcp** | 13.66 | -0.51 | -2.88 | 0.07 | 0.55 | —    | —    | —     | —     | 1.2  | 0.8  |
| **fcc** | 14.40 | -2.48 | -2.69 | 0.52 | 0.74 | 0.28 | 0.16 | -0.57 | -0.21 | 1.0  | 0.7  |


The positive sign of the DMI indicates favoring of right rotating spin spirals and the positive sign of K indicates an easy out-of-plane magnetization direction. The coefficients are given in meV.

Like Malottki et. al. we shall assume the magnetic moment to be $3.0 \mu_B$.

### System geometry

The Pd/Fe/Ir(111) layers have a triangular lattice with a lattice constant $a \approx 2.72 \AA$.

So we can choose the in-plane Bravais vectors
$$
\begin{aligned}
    a_1 &= a \left(1,0\right)\\
    a_2 &= a \left(\frac{1}{2},\frac{\sqrt{3}}{2}\right)
\end{aligned}
$$

In order to have fast-running simulations, we will use a small system size with $32 \times 32 \times 1$ cells.

# Step 1: Fill the spirit input file

**Task:** Open the `input_fcc.cfg` and `input_hcp.cfg` files and enter the hamiltonian parameters as well as the system geometry.

**Tips**:
Make sure to adjust:
 - the number of basis cell
 - the strength and direction of the uniaxial anisotropy
 - the Bravais vectors
 - the lattice constant
 - the magnetic moment
 - the Heisenberg pairs
 - the boundary conditions
 - the external magnetic field (**we want to simulate the fcc system at B=4T and the hcp system at B=1.5T**)


**Important**: Pay attention to the definition of the Hamiltonian!

Malottki et. al define
$$
\mathcal{H}_\text{exc} = \sum_{ij} J_{ij} \mathbf{m}_{i} \cdot \mathbf{m}_{j},
$$
while Spirit uses
$$
\mathcal{H}_\text{exc} = \sum_{i<j} J_{ij} \mathbf{m}_{i} \cdot \mathbf{m}_{j}.
$$
Think about how to adjust the $J_{ij}$ given by Malottki et. al. in order to be consistent with Spirit's convention! *The same applies to $D_{ij}$.*

# Step 2: Minimize the energy of the skyrmions 

After the input files have been adjusted, we can try to find a skyrmion! 

**Task**: Adjust the following code so that a suitable guess for an initial skyrmion configuration is inserted.
Also try out one of the more performant (but less stable solvers) by switching `simulation.SOLVER_VP` out for `simulation.SOLVER_LBFGS_OSO`!

**Tips:** 
 - The minimization should not take longer than a few seconds. If it takes significantly longer, either your input file or your initial guess for the skyrmion is wrong.
 - All functions in spirit take the `p_state` as their first argument
 - You can use the `configuration.plus_z` and `configuration.skyrmion` functions. First create a plus_z background configuration and then insert an ansatz for the skyrmion.
 - Depending on your initial configurations, the energy minimization might not lead to a skyrmion. Make sure to experiment with the parameters (radius and phase) of the initial skyrmion guess. 


In [ ]:
from spirit import state, configuration, simulation, io, parameters

lattices = ["hcp", "fcc"]

for lattice in lattices:

    input_file = f"./input/{lattice}.cfg"
    with state.State(input_file, quiet=True) as p_state:

        # ==== Start of your code ====
        raise NotImplementedError("Implement the initial skyrmion guess")
        # ==== End of your code ====

        # Set the convergence threshold to 1e-8 meV
        parameters.llg.set_convergence(p_state, 1e-8)

        # The `n_iterations` argument gives the maximum number of iterations. This minimization should converge way before.
        simulation_info = simulation.start(p_state, method_type=simulation.METHOD_LLG, solver_type=simulation.SOLVER_VP, n_iterations=2000)

        # Print out the achieved torque and how many iterations it took.
        print(f"Reached a torque of {simulation_info.max_torque} meV after {simulation_info.total_iterations} iterations")

        io.image_write(p_state, f"output/skyrmion_minimized_{lattice}.ovf")

### 2.1 Plot the minimized Skyrmions

Here, we plot the spin directions in a certain radius around the center of the system with `matplotlib`. If your initial guess for the skyrmion was correct you should see a large skyrmion for the `fcc` stacking parameters and a smaller one for the `hcp` stacking parameters.

**Task:** Use the `io.image_read` function to load the previously minimized and saved spin configurations. You should obtain two skyrmions, both with a radius of around ~12 Angstrom.

**Tips:** If you have installed the `spirit` GUI, you may also visualize the spins by using the terminal command
```bash
spirit -f input_fcc.cfg -i skyrmion_minimized_fcc.ovf
```

In [ ]:
from spirit import state, io, geometry, system
import numpy as np
import matplotlib.pyplot as plt
from plotting import plot_spins_2d

import post_processing

fig, axes = plt.subplots(1,2)
lattices = ["hcp", "fcc"]

PLOT_RADIUS = 30

for ax, lattice in zip(axes, lattices):

    input_file = f"./input/{lattice}.cfg"
    with state.State(input_file, quiet=True) as p_state:

        # ==== Start of your code ====
        raise NotImplementedError("Implement reading in the .ovf file")
        # ==== End of your code ====

        spins = system.get_spin_directions(p_state)
        positions = geometry.get_positions(p_state)
        center = np.mean(positions, axis=0)

        mask = np.linalg.norm(positions - center, axis=1) < PLOT_RADIUS

        ax.set_title(lattice)
        plot_spins_2d(
            ax, positions[mask], spins[mask], angles="xy", scale_units="xy", scale=0.25, width=0.008
        )

        radius = post_processing.skyrmion_radius(positions, spins, center)
        print(f"Radius of {lattice} skyrmion: {radius} A")

fig.tight_layout()

# Save a plot to show your parents :)
fig.savefig("output/minimized_skyrmions.png", dpi=300)

# Step 3: Find the energy barriers for the skyrmion collapse to the ferromagnetic ground state

We are now equipped to find the energy barriers for the collapse of skrymions to the ferromagnetic ground state (+z). We will do so with the GNEB method, which computes a minimum energy path.

To run the GNEB method, we need to
1. Specify an initial image (we will use the skyrmions we have found via minimization)
2. Specify a final image (we will use the +z ground state)
3. Generate a number of interpolating images (we will use a uniform interpolation, provided by the `transition` module, but in principle the configuration of all intermediate images can be supplied manually as well)
4. Use the `simulation.start` method and supply the GNEB as a `method_type` together with a suitable solver.

**Task 1:** Adjust the code snippet below to create the initial chain

**Tips:**
- Read in the `f"skyrmion_{lattice}.ovf"` files with the `io.image_read` method
- Use the `chain.set_length` method so set the length of the current chain to `N_IMAGES`
- Create a `plus_z` configuration in the final image (You can use `configuration.plus_z` with an additional `idx_image=N_IMAGES-1` argument)
- Create the interpolation by using `transition.homogeneous(pstate, idx_1=0, idx_2=N_IMAGES-1)`

**Task 2:** The GNEB chains converge slowly and can be quite unstable, this means we cannot easily use the more powerful L-BFGS solvers. Here, we pre-relax the energy of the chain with the VP solver and then switch to the L-BFGS solvers. Adjust the code such that it runs the GNEB with
- a convergence criterion of `1e-8 meV`
- the `SOLVER_LBFGS_OSO` solver
- run the cell (finding the MEPs should not take more than a few minutes)

**Tips:**
- Set the convergence criterion in the same way as we do for the VP solvers
- the solver is supplied 

In [ ]:
from spirit import state, io, simulation, io, parameters, chain, transition

lattices = ["hcp", "fcc"]

N_IMAGES = 12  # We use this many images (including the endpoints)

for ax, lattice in zip(axes, lattices):
    input_file = f"./input/{lattice}.cfg"

    with state.State(input_file, quiet=True) as p_state:

        # Creation of the initial chain
        # ==== Start of your code ====
        raise NotImplementedError("Implement the construction of the initial chain")
        # ==== End of your code ====

        # Save the initial chain to a file
        io.chain_write(p_state, f"output/chain_initial_{lattice}.ovf")

        # Relax the chain with a loose convergence criterion and the VP solver
        # This step will help us to find which image to use as the climbing image
        parameters.gneb.set_convergence(p_state, 1e-3)
        simulation_info = simulation.start(
            p_state, method_type=simulation.METHOD_GNEB, solver_type=simulation.SOLVER_VP, n_iterations=5000
        )
        io.chain_write(p_state, f"output/chain_before_ci_{lattice}.ovf")
        print(
            f"Reached a torque of {simulation_info.max_torque} meV after {simulation_info.total_iterations} iterations"
        )

        # Tighten the convergence criterion and activate the climbing image method
        parameters.gneb.set_convergence(p_state, 1e-3)
        # This will change the image_type of the highest energy image to climbing image
        parameters.gneb.set_image_type_automatically(p_state)
        simulation_info = simulation.start(
            p_state, method_type=simulation.METHOD_GNEB, solver_type=simulation.SOLVER_VP, n_iterations=5000
        )
        print(
            f"Reached a torque of {simulation_info.max_torque} meV after {simulation_info.total_iterations} iterations"
        )

        # ==== Start of your code ====
        raise NotImplementedError(
            "Implement running the GNEB method with the `simulation.SOLVER_LBFGS_OSO` solver and a convergence threshold of 1e-8"
        )
        # ==== End of your code ====

        print(
            f"Reached a torque of {simulation_info.max_torque} meV after {simulation_info.total_iterations} iterations"
        )

        io.chain_write(p_state, f"output/chain_final_{lattice}.ovf")

        energy_images = chain.get_energy(p_state)
        idx_sp = np.argmax(energy_images)
        energy_barrier = energy_images[idx_sp] - energy_images[0]
        print(f"The energy barrier is {energy_barrier = } meV. The index of the saddle point is {idx_sp}")

### 3.1 Plot the minimum energy paths
Now it's time to plot our minimum energy paths. The cell below plots the MEPs (before application of the climbing image).

**Task:** Change the code snipped so that it plots the final chain instead. How do the energy barriers change? Compare to Malottki et. al  Figure 3.

**Tips:** Just change the file, which is read in by `io.chain_read`

In [ ]:
from spirit import state, io, chain, simulation, parameters
import matplotlib.pyplot as plt

lattices = ["hcp", "fcc"]

fig, axes = plt.subplots(1, 2)

for ax, lattice in zip(axes, lattices):
    input_file = f"./input/{lattice}.cfg"

    with state.State(input_file, quiet=True) as p_state:

        # ==== Start of your code ====
        io.chain_read(p_state, f"output/chain_before_ci_{lattice}.ovf") # <--- Change this
        # ==== End of your code ====

        parameters.gneb.set_image_type_automatically(p_state)
        simulation.start(
            p_state, simulation.METHOD_GNEB, simulation.SOLVER_VP, n_iterations=1
        )

        energy_images = np.array(chain.get_energy(p_state))
        reaction_coordinate_images = np.array(chain.get_reaction_coordinate(p_state))
        energy_interpolated = np.array(chain.get_energy_interpolated(p_state))
        reaction_coordinate_interpolated = np.array(
            chain.get_reaction_coordinate_interpolated(p_state)
        )

        ax.axhline(0, ls="--", color="grey")
        ax.set_title(lattice)
        ax.plot(
            reaction_coordinate_interpolated,
            energy_interpolated - energy_images[0],
            marker="None",
        )
        ax.plot(
            reaction_coordinate_images,
            energy_images - energy_images[0],
            marker=".",
            color="black",
            ls="None",
        )

        energy_barrier = np.max(energy_interpolated) - energy_images[0]
        ax.axhline(energy_barrier, label=f"$\\Delta E$ = {energy_barrier:.1f} meV", ls="-", color="grey")

        ax.legend()
        ax.set_xlabel("reaction coordinate [rad]")
        ax.set_ylabel("energy [meV]")

fig.tight_layout()
fig.savefig("output/gneb.png", dpi=300)